# A Machine Learning Model
## Applied to Voting Behavior in Arizona
### Keras Neural Network 


Below I just load a bunch of dependencies. I follow this with a GBQ query to get the data. I then do some data cleaning. Finally, I split the data into train and test sets.    

A political engagement indicator was created, scored 1 if the voter participated in the 2020 primary, as well as the 2018 primary and general elections. 


In [14]:
#import tensorflow as tf
import os
import pandas as pd
from datetime import datetime
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/Chris/Dropbox/Keys/az-voter-file-30395362c45b.json"


In [15]:
import pandas_gbq
from google.cloud import bigquery
from sklearn.preprocessing import MinMaxScaler
import pandas_gbq


Always manage your python environments. I had to create a special one here because of my M1 chip. Don't ask me why, I don't know why tensorflow requires a separate build, but it does. For me it requires

**pip install tensorflow-macos==2.4.1**

**pip install tensorflow-metal==0.1.1**

But first -- and this is important -- create a virtual environment. There are a lot of conflicts that arise, and it's just easiest to keep separate spaces. Something like this:

**python3 -m venv "/Users/Chris/website_fall22/site/tensorflow"**

**source "/Users/Chris/website_fall22/site/tensorflow/bin/activate"**

Then... pip install upgrade and pip install all the packages, like pandas, numpy, sklearn, etc.


I query the BQ below.

In [16]:
### Formulate the SQL query to pull the data from BigQuery

query = """
    SELECT 
    geo_id,
    registrant_id,
    general_2020,
    primary_2020,
    general_2018,
    primary_2018,
    general_2016,
    primary_2016,
    general_2014,
    bachelors_degree,
    total_pop,
    birth_year,
    registration_change,
    median_age,
    median_income,
    white_pop,
    black_pop,
    asian_pop,
    hispanic_pop,
    amerindian_pop,
    housing_units,
    employed_pop,
    armed_forces,
    pop_in_labor_force,
FROM `az-voter-file.registration.clean_data_machine_learning_blocks`
"""

df = pandas_gbq.read_gbq(query, project_id="az-voter-file")
df.to_pickle('voter_file00_00_02.pkl') ## For later load, not to sync.

Downloading: 100%|██████████| 3895283/3895283 [08:34<00:00, 7578.32rows/s]


This is pretty rudimentary, and likely overkill, but we can compare it to far simpler measures as well. I created a variable, called "engaged, that is 1 if the voter participated in the 2020 primary, as well as the 2018 primary and general elections. I then split the data in half, into a test set and a train set. I then train a model on the train set, and test it on the test set. The model I use is a "neural network" with an input layer, 4 hidden layers, and an output layer. I tested this, specifying different parameterizations and hidden layers. It really doesn't matter. I hit about 85% accuracy, which is marginal, but far better than chance for these data. The features I use to train the model are, primary and general election voting prior to 2018, as well as the following characteristics measured at the  characteristics:
_________
### Voter Level Information
______
* 'general_2016', 
* 'primary_2016', 
* 'general_2014', 
* 'registration_change',
* 'registration_date',
_________
### Block level Information
_________
* 'poverty',
* 'age', 
* 'median_age', 
* 'median_income',
* 'white_pop', 
* 'black_pop', 
* 'asian_pop', 
* 'hispanic_pop', 
* 'amerindian_pop', 
* 'gini_index', 
* 'housing_units', 
* 'employed_pop' 

All variables were 0-1 standardized prior to analysis.



In [17]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

import keras
from keras import layers
#from tensorflow.keras import layers

#"https://caffeinedev.medium.com/how-to-install-tensorflow-on-m1-mac-8e9b91d93706"

df = pd.read_pickle('voter_file00_00_02.pkl')


reg_change=  (pd.to_datetime("11-04-2020", format = "%m-%d-%Y") - pd.to_datetime(df['registration_change'], 
                                format = "%Y-%m-%d", errors = 'coerce'))
df["registration_change"] = reg_change.dt.days 



st_dat = df[['general_2020', 'primary_2020', 'general_2018', 
    'primary_2018', 'general_2016', "bachelors_degree",
    'primary_2016', 'general_2014',
    'registration_change',
    'median_age', 'median_income',
    'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 
    'amerindian_pop', 'housing_units', 
    'employed_pop']]
 
st_dat = st_dat.dropna(how = 'any')
scaler = MinMaxScaler()
st_dat_array = scaler.fit_transform(st_dat)
st_dat = pd.DataFrame(st_dat_array, columns = st_dat.columns)


st_dat['engaged'] = np.where((((st_dat["primary_2018"] == 1)  
                                     & (st_dat["primary_2020"] == 1) 
                                     & (st_dat["general_2018"] == 1)
                                    )),1,0) 
train, test = train_test_split(st_dat, test_size=0.2)

features_train   =  train[['general_2016',  "bachelors_degree", 
                            'primary_2016', 'general_2014', 
                            'registration_change',
                            'median_age', 'median_income',
                            'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 
                            'amerindian_pop', 'housing_units', 
                            'employed_pop']]
labels_train     =  pd.DataFrame({"engaged":train['engaged'],  "not_engaged":1-train['engaged']})


features_train_array = np.array(features_train, np.float64)
labels_train_array   = np.array(labels_train,   np.float64)


features_test   = test[['general_2016',  "bachelors_degree", 
                            'primary_2016', 'general_2014', 
                            'registration_change',
                            'median_age', 'median_income',
                            'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 
                            'amerindian_pop', 'housing_units', 
                            'employed_pop']]
labels_test     =  pd.DataFrame({"engaged":test['engaged'],  "not_engaged":1-test['engaged']})


features_test_array = np.array(features_test, np.float64)
labels_test_array   = np.array(labels_test,   np.float64)

from tensorflow.keras.regularizers import l1_l2
model = tf.keras.Sequential()
# Define the first layer
model.add(keras.layers.Dense(20, activation='softmax', 
                               input_shape=(features_train.shape[1],)))
model.add(keras.layers.Dropout(0.25))
# model.add(keras.layers.Dense(10, activation='softmax'))
# model.add(keras.layers.Dense(5, activation='softmax'))
model.add(keras.layers.Dense(2, activation='softmax'))
    

# Finish the model compilation
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [23]:
# #! pip install tensorflow-macos --upgrade 
# import tensorflow as tf
# !pip install keras

In [19]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

model.fit(features_train_array, 
          labels_train_array, epochs=10, batch_size=1000, 
          validation_split=0.20)

TensorFlow version: 2.9.2
Num GPUs Available:  1
Epoch 1/10
1967/1967 [==============================] - 20s 10ms/step - loss: 0.3922 - accuracy: 0.8384 - val_loss: 0.3822 - val_accuracy: 0.8442
Epoch 2/10
1967/1967 [==============================] - 19s 10ms/step - loss: 0.3908 - accuracy: 0.8401 - val_loss: 0.3819 - val_accuracy: 0.8442
Epoch 3/10
1967/1967 [==============================] - 19s 10ms/step - loss: 0.3900 - accuracy: 0.8412 - val_loss: 0.3817 - val_accuracy: 0.8442
Epoch 4/10
1967/1967 [==============================] - 19s 10ms/step - loss: 0.3894 - accuracy: 0.8416 - val_loss: 0.3818 - val_accuracy: 0.8442
Epoch 5/10
1967/1967 [==============================] - 19s 10ms/step - loss: 0.3892 - accuracy: 0.8417 - val_loss: 0.3815 - val_accuracy: 0.8441
Epoch 6/10
1967/1967 [==============================] - 19s 10ms/step - loss: 0.3889 - accuracy: 0.8417 - val_loss: 0.3814 - val_accuracy: 0.8442
Epoch 7/10
1967/1967 [==============================] - 19s 10ms/step - los

Above, where I constructed the training data, I also set aside 20 percent of the sample. **The model was not trained on this model. These are fresh data, randomly drawn, so that we can compare the outcome to the predicted outcome.** Overall, I reach about 85% acccuracy, which is not great, but far better than chance.

In [20]:
features_test   =   test[['general_2016',  "bachelors_degree",
                            'primary_2016', 'general_2014',   
                            'registration_change',
                            'median_age', 'median_income',
                            'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 
                            'amerindian_pop', 'housing_units', 
                            'employed_pop']]
labels_test     =  pd.DataFrame({"engaged":test['engaged'],  "not_engaged":1-test['engaged']})
features_test_array = np.array(features_test, np.float64)
labels_test_array   = np.array(labels_test,   np.float64)

In [21]:
outcome = model.predict(features_test_array) > 0.5


   81/19204 [..............................] - ETA: 36s

2022-09-25 14:36:44.275245: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19204/19204 [==============================] - 32s 2ms/step


I've tinkered with the model quite a bit. I can't seem to improve it. It's not a remarkable degree of accuracy, but there's really not all that much individual level data, so I'm not sure.

In [22]:
preds1 = model.predict(features_test_array) > 0.5
preds2 = model.predict(features_train_array)

# Evaluate the model
from tensorflow.keras.metrics import Accuracy, Precision, Recall
acc = Accuracy()
prec = Precision()
recall = Recall()
acc.update_state(labels_test_array, preds1)

acc.result().numpy()
# prec.result().numpy()
# recall.result().numpy()

76815/76815 [==============================] - 119s 2ms/step


0.8423927

In [23]:
### Standardize the data, train with variables below ####
full_data = df[['registrant_id', 'general_2020', 'primary_2020', 'general_2018', 
    'primary_2018', 'general_2016', "bachelors_degree",
    'primary_2016', 'general_2014',
    'registration_change',
    'median_age', 'median_income',
    'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 
    'amerindian_pop',  'housing_units', 
    'employed_pop']]
full_data = full_data.dropna(how = 'any')
registrant_id = full_data['registrant_id']
full_data_array = scaler.fit_transform(full_data)
full_data = pd.DataFrame(full_data_array, columns = full_data.columns)
full_data['engaged'] = np.where((((full_data["primary_2018"] == 1)  
                                     & (full_data["primary_2020"] == 1) 
                                     & (full_data["general_2018"] == 1)
                                    )),1,0) 
labels_full     =  pd.DataFrame({"engaged": full_data['engaged'],  "not_engaged": 1-full_data['engaged']})
features_full    =  full_data[['general_2016', "bachelors_degree",
    'primary_2016', 'general_2014', 
    'registration_change',
    'median_age', 'median_income',
    'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 
    'amerindian_pop',  'housing_units', 
    'employed_pop']]

features_full_array = np.array(features_full, np.float64)
labels_full_array   = np.array(labels_full,   np.float64)


In [24]:
preds = model.predict(features_full_array)
preds = pd.DataFrame(preds)
preds.head()


96019/96019 [==============================] - 156s 2ms/step


,0,1
0,0.030005,0.969995
1,0.260459,0.739541
2,0.054661,0.945339
3,0.281141,0.718859
4,0.133359,0.866641


In [45]:
np.random.binomial(1, preds.iloc[:,0])

array([0, 0, 0, ..., 1, 1, 0])

In [11]:
upload_data = pd.DataFrame( {"engaged_pr" : preds.iloc[:,0], 
                    "not_engaged_pr" : preds.iloc[:,1],  
                    "point" : np.random.binomial(1, preds.iloc[:,0]),
                    "engaged_true" :  full_data["engaged"],
                    "registrant_id" : registrant_id.tolist() } )

upload_data.head()

,engaged_pr,not_engaged_pr,point,engaged_true,registrant_id
0,0.735514,0.264486,0,1,23265651
1,0.254172,0.745828,0,0,21978552
2,0.048989,0.951011,0,0,27152091
3,0.300747,0.699253,1,0,21951627
4,0.359372,0.640628,0,0,21682575


In [25]:
from google.cloud import bigquery
from sklearn.preprocessing import MinMaxScaler
import pandas_gbq
bqclient = bigquery.Client()
project_id = "az-voter-file"
pandas_gbq.to_gbq(upload_data, "az-voter-file.registration.nn05", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 5592.41it/s]
